In [36]:
from pathlib import Path
from data_utils.frame_dataset import FrameDataset
import lightning as L
import polars as pl
from torch.utils.data import DataLoader
from models.lightning_wrapper import LightningWrapper

model_path = "ckpts/mobilenetv3_large_100.miil_in21k_ft_in1k.ckpt"

model = LightningWrapper.load_from_checkpoint(model_path)
trainer = L.Trainer(accelerator="cpu")

paths = list(Path("1863051677").rglob("*.jpg"))[5_000:6_000]
df = pl.DataFrame(
    {"path": paths, "frame": [int(p.stem.removeprefix("img")) for p in paths]}
)
df = df.sort("frame")

ds = FrameDataset(df, model.get_transforms(is_training=False), 1, is_train=False)
dls = DataLoader(ds, batch_size=32, num_workers=2, pin_memory=True)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [37]:
import numpy as np
import torch

preds_list: list[torch.Tensor] = trainer.predict(model, dataloaders=dls)  # type: ignore
preds = torch.cat(preds_list)
pred_class = torch.argmax(preds, dim=1)
preds_class = np.repeat(pred_class.numpy(), ds.frames_per_clip)

Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [44]:
import torchvision.models as mls
model = mls.resnet50(pretrained=True)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.


In [46]:
import shap
batch = next(iter(dls))
e = shap.DeepExplainer(model, batch[:10].clone())

In [48]:
shap_values = e.shap_values(batch[:3].clone())

RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.

# ---


In [16]:
np.array(batch[1]).shape

(3, 224, 224)

In [40]:
from lime import lime_image
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(batch[1]).transpose(1, 2, 0), 
                                         lambda x: trainer.predict(model, torch.from_numpy(x.transpose(0, 3, 1, 2))), # classification function
                                         top_labels=2, 
                                         hide_color=0, 
                                         batch_size=1,
                                         num_samples=100)

  0%|          | 0/100 [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

AssertionError: expected 4D input (got 3D input)

In [12]:
import shap


topk = 2
batch_size = 50
n_evals = 10000

# define a masker that is used to mask out partitions of the input image.
masker_blur = shap.maskers.Image("blur(128,128)", ds[0].shape)

# create an explainer with model and image masker
explainer = shap.Explainer(lambda x: trainer.predict(model, torch.Tensor(x).unsqueeze(0)),
                           masker_blur,
                           output_names=["Nothing", "Highlight"])

# feed only one image
# here we explain two images using 100 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(
    [ds[0]],
    max_evals=n_evals,
    batch_size=batch_size,
    outputs=shap.Explanation.argsort.flip[:topk],
)

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for axis 0 with size 1